In [1]:
from fast_feature.datasets.titanic import load_titanic
import polars as pl
import pandas as pd
import numpy as np
from fast_feature.dataframe_checks import *

In [2]:
df = load_titanic()
df_pl = pl.from_pandas(df)

In [11]:
from fast_feature.encoding import OrdinalEncoder
import polars as pl

df = {
        "var_A": ["A"] * 6 + ["B"] * 10 + ["C"] * 4,
        "var_B": ["A"] * 10 + ["B"] * 6 + ["C"] * 4,
        "target": [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
    }
df_enc = pl.DataFrame(df)

In [ ]:
df = {
        "var_A": ["B"] * 9 + ["A"] * 6 + ["C"] * 4 + ["D"] * 1,
        "var_B": ["A"] * 10 + ["B"] * 6 + ["C"] * 4,
        "target": [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
    }
df_enc_rare = pl.DataFrame(df)

In [12]:
encoder = OrdinalEncoder(unseen="ignore")
encoder.fit(df_enc[["var_A", "var_B"]], df_enc["target"])
encoder.transform(df_enc_rare[["var_A", "var_B"]])

NameError: name 'df_enc_rare' is not defined

In [7]:
df = {
        "var_A": ["B"] * 9 + ["A"] * 6 + ["C"] * 4 + ["D"] * 1,
        "var_B": ["A"] * 10 + ["B"] * 6 + ["C"] * 4,
        "target": [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
    }
df = pd.DataFrame(df)
df.loc[0, "var_A"] = np.nan

In [5]:
encoder = OrdinalEncoder(encoding_method="arbitrary")

In [10]:
encoder.fit(df)

ValueError: Some of the variables in the dataset contain NaN. Check and remove those before using this transformer or set the parameter `missing_values='ignore'` when initialising this transformer.

In [6]:
from fast_feature.encoding import OneHotEncoder

In [9]:
oh = OneHotEncoder()

In [12]:
oh.fit(df_pl)

AttributeError: 'DataFrame' object has no attribute 'select_dtypes'

In [11]:
df

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,NaN,C,NaN,328,NaN
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,NaN,C,NaN,304,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN


In [44]:
isinstance(y.dtype, pl.String)

False

In [74]:
y = df[['survived']]

y2 = df_pl[['survived']]


In [62]:
y2.to_series().is_null().any()

np.True_

In [71]:
type(df[['survived']].squeeze())\

pandas.core.series.Series

In [105]:
from typing import List, Tuple, Union

import numpy as np
import pandas as pd
from scipy.sparse import issparse
from sklearn.utils.validation import _check_y, check_consistent_length, column_or_1d
import polars as pl


def check_y(
    y: Union[np.generic, np.ndarray, pl.Series, pl.DataFrame, List, pd.DataFrame, pd.Series],
    y_numeric: bool = False,
) -> pl.Series:
    """
    Checks that y is a series or a dataframe, or alternatively, if it can be converted
    to a series.

    Parameters
    ----------
    y : pl.Series, pl.DataFrame, np.array, list
        The input to check and copy or transform.

    y_numeric : bool, default=False
        Whether to ensure that y has a numeric type. If dtype of y is object,
        it is converted to float64. Should only be used for regression
        algorithms.

    Returns
    -------
    y: pl.Series
    """


    if  isinstance(y, pl.DataFrame):
        if y.shape[1] == 1:
            y = y.to_series()
        else:
            raise ValueError("Please make sure you are using 1 dimension for Y")
        
    if isinstance(y, pd.DataFrame):
        if y.shape[1] == 1:
            y = y.squeeze(axis=0)
        else:
            raise ValueError("Please make sure you are using 1 dimension for Y")
        
    
    if y is None:
        raise ValueError(
            "requires y to be passed, but the target y is None",
            "Expected array-like (array or non-string sequence), got None",
            "y should be a 1d array",
        )

    elif isinstance(y, pl.Series):
        if y.is_null().any():
            raise ValueError("y contains NaN values.")
        if isinstance(y.dtype, pl.String) == False and not np.isfinite(np.array(y)).all():
            raise ValueError("y contains infinity values.")
        if y_numeric and isinstance(y.dtype, pl.String) :
            y = y.cast(float)
        y_cp = y.clone()

    elif isinstance(y, pd.Series):
        if y.isnull().any():
            raise ValueError("y contains NaN values.")
        if y.dtype != "O" and not np.isfinite(y).all():
            raise ValueError("y contains infinity values.")
        if y_numeric and y.dtype == "O":
            y = y.astype("float")
        y_cp = pl.from_pandas(y)
    else:
        try:
            y_cp = column_or_1d(y)
            y_cp = _check_y(y_cp, multi_output=False, y_numeric=y_numeric)
            y_cp = pl.Series(y_cp).clone()
        except:
            raise ValueError("Please make sure you are using 1 dimension for Y")
    return y_cp


In [92]:
check_y(df_pl['survived'])

survived
i64
1
1
0
0
0
…
0
0
0


In [93]:
check_y(df['survived'])

survived
i64
1
1
0
0
0
…
0
0
0


In [104]:
check_y(np.array([1, 1, 1, 1]))

""
i64
1
1
1
1


In [100]:
y_numeric = True
_check_y(y_cp, multi_output=False, y_numeric=y_numeric)

array([1, 1, 1, 1])

In [109]:
from polars.testing import assert_frame_equal

In [110]:
pl.testing.assert_frame_equal

<function polars.testing.asserts.frame.assert_frame_equal(left: 'DataFrame | LazyFrame', right: 'DataFrame | LazyFrame', *, check_row_order: 'bool' = True, check_column_order: 'bool' = True, check_dtypes: 'bool' = True, check_exact: 'bool' = False, rtol: 'float' = 1e-05, atol: 'float' = 1e-08, categorical_as_str: 'bool' = False) -> 'None'>

In [111]:
pl.testing.assert_frame_equal

<function polars.testing.asserts.frame.assert_frame_equal(left: 'DataFrame | LazyFrame', right: 'DataFrame | LazyFrame', *, check_row_order: 'bool' = True, check_column_order: 'bool' = True, check_dtypes: 'bool' = True, check_exact: 'bool' = False, rtol: 'float' = 1e-05, atol: 'float' = 1e-08, categorical_as_str: 'bool' = False) -> 'None'>

In [3]:
from typing import List, Optional, Union

import numpy as np
import pandas as pd
import polars as pl

from fast_feature._docstrings.fit_attributes import (
    _feature_names_in_docstring,
    _n_features_in_docstring,
    _variables_attribute_docstring,
)
from fast_feature._docstrings.init_parameters.all_trasnformers import (
    _variables_categorical_docstring,
)
from fast_feature._docstrings.init_parameters.encoders import _ignore_format_docstring
from fast_feature._docstrings.methods import _fit_transform_docstring
from fast_feature._docstrings.substitute import Substitution
from fast_feature.dataframe_checks import _check_contains_na, check_X
from fast_feature.encoding.base_encoder import (
    CategoricalInitMixin,
    CategoricalMethodsMixin,
)

NameError: name 'pl' is not defined

In [1]:
from fast_feature.encoding.base_encoder import CategoricalInitMixin

NameError: name 'pl' is not defined

In [2]:
from fast_feature.encoding.count_frequency import CountFrequencyEncoder

NameError: name 'pl' is not defined

In [2]:
from fast_feature._docstrings.fit_attributes import (
    _feature_names_in_docstring,
    _n_features_in_docstring,
    _variables_attribute_docstring,
)

NameError: name 'fast_feature' is not defined

In [2]:
import warnings
from typing import List, Union

import polars as pl
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted

from fast_feature._base_transformers.mixins import GetFeatureNamesOutMixin
from fast_feature._check_init_parameters.check_variables import (
    _check_variables_input_value,
)
from fast_feature._docstrings.init_parameters.all_trasnformers import (
    _missing_values_docstring,
    _variables_categorical_docstring,
)
from fast_feature._docstrings.init_parameters.encoders import _ignore_format_docstring
from fast_feature._docstrings.substitute import Substitution
from fast_feature.dataframe_checks import (
    _check_optional_contains_na,
    _check_X_matches_training_df,
    check_X,
)
from fast_feature.tags import _return_tags
from fast_feature.variable_handling import (
    check_all_variables,
    check_categorical_variables,
    find_all_variables,
    find_categorical_variables,
)

In [1]:
o